In [272]:
%load_ext autoreload

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import math

In [3]:
with open("../../Project 2/CS205_SP_2022_SMALLtestdata__44.txt", "r") as file:
    data= file.readlines()


In [16]:
dt_row=[]
for line in data:
    row=[]
    for col in line.split():
        row.append(float(col))
    dt_row.append(row)





In [26]:
df= pd.DataFrame(dt_row, columns=['target', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10'])

### Leave out one cross validation

In [40]:
cols= list(df.columns)
cols1= cols[1:]
cols1.append(cols[0])
df= df[cols1]

#all_vals = df[list(df.columns)[:-1]].values
#all_labels = df[list(df.columns)[-1]].values
#all_vals.shape, all_labels.shape

In [354]:
def compute_dist(test_pt, train_pt):
    dist_mat= np.round(math.sqrt(np.sum(np.abs(train_pt-test_pt)**2)), 3)
    return dist_mat

In [351]:
def knn_classifier(test_data, training_data, training_labels, K=1):
    pred_labels = np.zeros(len(test_data))
    #print("pred_labels: ", pred_labels)

    all_dists = np.zeros(len(training_data))
        #print("all_dists: ", all_dists.shape)
    count = 0
    for train_pt in training_data: #for every row in train data
        all_dists[count] = compute_dist(test_data, train_pt)   #np.linalg.norm(train_pt-test_pt) #compute_dist(test_pt, train_pt)  #
        #print(all_dists[count])
        count = count+1
    
    sorted_indices = np.argsort(all_dists)
    pred_labels = training_labels[sorted_indices[0]]
    #print(len(pred_labels))
    return pred_labels

In [355]:
# split data based on Leave out one cross validation
def loov(features_list):
    all_accuracy= []
    for i in range(len(df)):
        #if i%50==0:
        #    print(i)

        #train-test split
        test_df= df.iloc[[i, ]]
        train_df= pd.concat([df.iloc[:i, ], df.iloc[i+1: ,]])
        X_train, X_test, y_train, y_test= train_df.iloc[:, features_list] , test_df.iloc[:, features_list], train_df.iloc[:, -1], test_df.iloc[:, -1]
        #print("X_train: ", X_train.shape, X_test.shape, y_train.shape, y_test.shape)
        pred_labels = knn_classifier(X_test.values, X_train.values, y_train.values, 1)
        accuracy = sum(y_test.values == pred_labels)
        #print(pred_labels, y_test.values, accuracy, len(pred_labels))
        all_accuracy.append(accuracy)

    return np.array(all_accuracy).mean()
    

In [ ]:
#feature selection

